Welcome to ExoTiC-UVIS Stage 2! In this Stage, we will:

1. Load the data we reduced in Stage 1.
2. Calibrate the +1st and -1st order using GRIMSCONF.
3. Extract the 1D spectra from each order using an unweighted extraction.
4. Clean and align our cleaned spectra.
5. Save each spectrum to an xarray file.

Make sure to run the ExoTiC-UVIS Stage 0 and 1 notebooks before you run this one! This notebook relies on files that have been downloaded from Stage 0 and reduced in Stage 1.

To get started, run the next cell, which imports the packages we need and creates a directory for ExoTiC-UVIS to operate from. Then follow the next markdown cell for instructions on how to execute Stage 2.

In [1]:
import os
from exotic_uvis import run_pipeline

config_directory = 'hustle_config'
if not os.path.exists(config_directory):
    os.makedirs(config_directory)

GRISMCONF CONFIGURATION FILES

Before you can run Stage 2, you must download the HST WFC3/UVIS G280 configuration files provided by Pirzkal 2017. You can find these files here:

https://github.com/npirzkal/GRISM_WFC3/tree/master/UVIS

Download all files in this directory to your own computer. Then, copy the absolute path to the UVIS_G280_CCD2_V2.conf file, as you will need to tell ExoTiC-UVIS where this file is for it to be able to operate.

THE STAGE 2 .HUSTLE FILE

ExoTiC-UVIS operates using configuration files, designated with the .hustle file extension. Each .hustle file controls one Stage of ExoTiC-UVIS. .hustle files are more human-readable than raw code and they allow you to easily reproduce previous runs of ExoTiC-UVIS as well as rapidly share your reduction and analysis methods with your colleagues.

Now we need to make our Stage 2 .hustle file. The next cell contains a template stage_2.hustle file which you can modify to run Stage 2 of ExoTiC-UVIS. We're going to modify it so that it calibrates and extracts from the HUSTLE program observations of the hot Jupiter WASP-127b that we downloaded in Stage 0 and reduced in Stage 1. To make it do this, make the following changes to the .hustle file template:

1. Set input_run to the string 'reduction_1' so we can use our reduced files from the last Stage.
2. Set output_run to the string 'extraction_1' to keep our different extraction attempts separated.
3. Set the verbose, show_plots, and save_plots variables to each be 0, 1, or 2. verbose controls how many printed statements the pipeline produces, letting you keep track of what the pipeline is doing and what step it is on. show_plots allows the pipeline to temporarily pause execution to show you an interactive plot. save_plots saves output .png or .gif files for plots and other graphics produced. 0 prints/shows/saves nothing, while 2 prints/shows/saves everything.
4. Set path_to_cal to the absolute path of UVIS_G280_CCD2_V2.conf on your computer so that GRISMCONF knows where to find the files needed to calibrate the traces.
5. Set traces_to_conf to ('+1','-1'). This tells ExoTiC-UVIS that we want to extract the +1st and -1st orders.
6. We will be using the 'box' method of extraction, which extracts the data using an unweighted aperture. Set halfwidths_box to (12,12). This tells ExoTiC-UVIS to use an aperture halfwidth of 12 pixels for both orders.
7. Set outlier_sigma to 5.0 and align to True. This tells the pipeline to reject spectral outliers at 5.0-sigma, and to use cross-correlation to correct any displacements that may have happened during the observation.

 As you modify each variable, take a moment to read the comment to its right. These comments tell you what each variable does and what else you can do with them.

In [2]:
hustle_stage_2_file = f"""
# ExoTiC-UVIS config file for launching Stage 2: Extraction

# Setup for Stage 2
toplevel_dir    'tutorial'                                  # Directory where your current project files are stored. This folder should contain the specimages/, directimages/, etc. folders with your data as well as the outputs folder.
input_run       None                                        # Str. This is the name of the Stage 1 run you want to load.
output_run      None                                        # Str. This is the name to save the current run to. It can be anything that does not contain spaces or special characters (e.g. $, %, @, etc.).
verbose         2                                           # Int from 0 to 2. 0 = print nothing. 1 = print some statements. 2 = print every action.
show_plots      2                                           # Int from 0 to 2. 0 = show nothing. 1 = show some plots. 2 = show all plots.
save_plots      2                                           # Int from 0 to 2. 0 = save nothing. 1 = save some plots. 2 = save all plots.

# Step 1: Read in the data

# Step 2: Trace configuration
path_to_cal     './'                                        # Str. The absolute path to the .conf file used by GRISMCONF for the chip your data were taken on.
traces_to_conf  None                                        # Lst of str. The traces you want to configure and extraction from.
refine_fit      False                                       # Bool. If True, uses Gaussian fitting to refine the trace solution.

# Step 3: 1D spectral extraction
method          'box'                                       # Str. Options are 'box' (draw a box around the trace and sum without weights) or 'optimal' (weight using Horne 1986 methods).
correct_zero    False                                       # Bool. Whether to model the contaminating 0th order and subtract it from your data during extraction. Sometimes works, sometimes just adds lots of scatter.
subtract_contam False                                       # Bool. Whether to model the contaminating orders and subtract them from your trace during extraction. Sometimes works, sometimes just adds lots of scatter.
sens_correction False                                       # Bool. Whether to correct for the G280's changing sensitivity as a function of wavelength. Since absolute calibrated spectra aren't needed in exoplanetary sciences, you can skip this safely.
mask_objs       [[0,0,0],]                                  # List of lists. If there are background objects in your planned aperture, mask them here. Each entry is (x,y,radius).

# Step 3a: Box extraction parameters
determine_hw    False                                       # Bool. If True, automatically determines preferred half-width for each order by minimizing out-of-transit/eclipse residuals.
indices         ([0,10],[-10,-1])                           # Lst of lsts of int. If determine_hw, these are the indices used to estimate the out-of-transit/eclipse residuals.
halfwidths_box  (25,25)                                     # Lst of ints. The half-width of extraction aperture to use for each order. Input here is ignored if 'determine_hw' is True.

# Step 3b: Optimum extraction parameters
aperture_type   'median'                                    # Str. Type of aperture to draw. Options are 'median', 'polyfit', 'smooth', or 'curved_poly'.
halfwidths_opt  (12,12)                                     # Lst of ints. The half-width of extraction aperture to use for each order. For optimum extraction, you should make this big (>12 pixels at least). There is no 'preferred' half-width in optimum extraction due to the weights.

# Step 4: 1D spectral cleaning and aligning
outlier_sigma   None                                        # Float. Sigma at which to reject spectral outliers in time. Outliers are replaced with median of timeseries. Enter False to skip this step.
align           False                                       # Bool. If True, uses cross-correlation to align spectra to keep wavelength solution consistent.

# ENDPARSE
"""

# Now we write the contents of the config file out to a .hustle file.
with open(os.path.join(config_directory,'stage_2_input_config.hustle'), 'w') as f:
    f.write(hustle_stage_2_file)

In [3]:
hustle_stage_2_file = f"""
# ExoTiC-UVIS config file for launching Stage 2: Extraction

# Setup for Stage 2
toplevel_dir    'tutorial'                                  # Directory where your current project files are stored. This folder should contain the specimages/, directimages/, etc. folders with your data as well as the outputs folder.
input_run       'reduction_1'                               # Str. This is the name of the Stage 1 run you want to load.
output_run      'extraction_1'                              # Str. This is the name to save the current run to. It can be anything that does not contain spaces or special characters (e.g. $, %, @, etc.).
verbose         2                                           # Int from 0 to 2. 0 = print nothing. 1 = print some statements. 2 = print every action.
show_plots      0                                           # Int from 0 to 2. 0 = show nothing. 1 = show some plots. 2 = show all plots.
save_plots      1                                           # Int from 0 to 2. 0 = save nothing. 1 = save some plots. 2 = save all plots.

# Step 1: Read in the data

# Step 2: Trace configuration
path_to_cal     '/Users/abbyboehm/opt/anaconda3/github_repos/ExoTiC-UVIS/G280_config/GRISM_WFC3/UVIS/UVIS_G280_CCD2_V2.conf'                                        # Str. The absolute path to the .conf file used by GRISMCONF for the chip your data were taken on.
traces_to_conf  ('+1','-1')                                 # Lst of str. The traces you want to configure and extraction from.
refine_fit      False                                       # Bool. If True, uses Gaussian fitting to refine the trace solution.

# Step 3: 1D spectral extraction
method          'box'                                       # Str. Options are 'box' (draw a box around the trace and sum without weights) or 'optimal' (weight using Horne 1986 methods).
correct_zero    False                                       # Bool. Whether to model the contaminating 0th order and subtract it from your data during extraction. Sometimes works, sometimes just adds lots of scatter.
subtract_contam False                                       # Bool. Whether to model the contaminating orders and subtract them from your trace during extraction. Sometimes works, sometimes just adds lots of scatter.
sens_correction False                                       # Bool. Whether to correct for the G280's changing sensitivity as a function of wavelength. Since absolute calibrated spectra aren't needed in exoplanetary sciences, you can skip this safely.
mask_objs       [[0,0,0],]                                  # List of lists. If there are background objects in your planned aperture, mask them here. Each entry is (x,y,radius).

# Step 3a: Box extraction parameters
determine_hw    False                                       # Bool. If True, automatically determines preferred half-width for each order by minimizing out-of-transit/eclipse residuals.
indices         ([0,10],[-10,-1])                           # Lst of lsts of int. If determine_hw, these are the indices used to estimate the out-of-transit/eclipse residuals.
halfwidths_box  (12,12)                                     # Lst of ints. The half-width of extraction aperture to use for each order. Input here is ignored if 'determine_hw' is True.

# Step 3b: Optimum extraction parameters
aperture_type   'median'                                    # Str. Type of aperture to draw. Options are 'median', 'polyfit', 'smooth', or 'curved_poly'.
halfwidths_opt  (12,12)                                     # Lst of ints. The half-width of extraction aperture to use for each order. For optimum extraction, you should make this big (>12 pixels at least). There is no 'preferred' half-width in optimum extraction due to the weights.

# Step 4: 1D spectral cleaning and aligning
outlier_sigma   5.0                                         # Float. Sigma at which to reject spectral outliers in time. Outliers are replaced with median of timeseries. Enter False to skip this step.
align           True                                        # Bool. If True, uses cross-correlation to align spectra to keep wavelength solution consistent.

# ENDPARSE
"""

# Now we write the contents of the config file out to a .hustle file.
with open(os.path.join(config_directory,'stage_2_input_config.hustle'), 'w') as f:
    f.write(hustle_stage_2_file)

Now that our config file is ready, simply use the cell below to execute Stage 2 of the pipeline!

This Stage requires no user interaction unless show_plots is set to greater than 0, which prompts ExoTiC-UVIS to interrupt execution to show the user the plots being generated in an interactive format. If you have set show_plots to 0, then the pipeline will automatically finish running in about 30 seconds.

In [ ]:
run_pipeline(config_files_dir=config_directory,
             stages=(2,))

You made it! I hope there were no problems with execution. Now let's check out the outputs!

Inside the 'tutorial/outputs/' directory should now be 'stage2/extraction_1/'. Inside that folder you will find:
- specs_+1.nc and specs_-1nc, which are xarray files containing all of your extracted spectra
- stage_2_reduction_1.hustle, a copy of the config file you used to run this extraction
- plots/, a folder full of diagnostic plots that let you know how extraction operated

You should check the s2_1Dspec_order+1 and -1 .gif files to make sure extraction was successful. Also take a look at the 2Dspec images. The former plots the native resolution light curves as a 2D image. Notice the bluest light curves tend to have more scatter, and the reddest light curves are the dimmest.

Take a look at the rawwlc images as well. The white light curves in these files may look a little strange. Don't panic - there is nothing wrong with your reduction technique! These are Hubble Space Telescope systematics and there are many methods out there for treating these (stay tuned for Stages 3-5!). The -1 light curve in particular is a great showcase of the exponential ramp, a systematic trend spanning each orbit which appears when charge in the CCD pixels does not fully drain between exposures.

Next, let's see what happens when you use the optimal extraction method of Horne 1986. This technique applies weights to the trace so that the contribution from brighter, less noisy pixels is emphasized over the contributions from dimmer, more noisy pixels. Copy your config cell into the cell below and make the following change:

1. Set output_run to 'extraction_2' so we can compare our different extraction techniques.
2. Set method to 'optimal' so we can use the optimal extraction technique.
3. Set aperture_type to 'median' and halfwidths_opt to (20,20). The former tells the pipeline to build the extraction spatial profile using the median of all frames. When doing optimal extraction, it is fine to use a large halfwidth - the weighting will take care of the noise.

In [5]:
hustle_stage_2_file = f"""
# ExoTiC-UVIS config file for launching Stage 2: Extraction

# Setup for Stage 2
toplevel_dir    'tutorial'                                  # Directory where your current project files are stored. This folder should contain the specimages/, directimages/, etc. folders with your data as well as the outputs folder.
input_run       'reduction_1'                               # Str. This is the name of the Stage 1 run you want to load.
output_run      'extraction_2'                              # Str. This is the name to save the current run to. It can be anything that does not contain spaces or special characters (e.g. $, %, @, etc.).
verbose         2                                           # Int from 0 to 2. 0 = print nothing. 1 = print some statements. 2 = print every action.
show_plots      0                                           # Int from 0 to 2. 0 = show nothing. 1 = show some plots. 2 = show all plots.
save_plots      1                                           # Int from 0 to 2. 0 = save nothing. 1 = save some plots. 2 = save all plots.

# Step 1: Read in the data

# Step 2: Trace configuration
path_to_cal     '/Users/abbyboehm/opt/anaconda3/github_repos/ExoTiC-UVIS/G280_config/GRISM_WFC3/UVIS/UVIS_G280_CCD2_V2.conf'                                        # Str. The absolute path to the .conf file used by GRISMCONF for the chip your data were taken on.
traces_to_conf  ('+1','-1')                                 # Lst of str. The traces you want to configure and extraction from.
refine_fit      False                                       # Bool. If True, uses Gaussian fitting to refine the trace solution.

# Step 3: 1D spectral extraction
method          'optimal'                                   # Str. Options are 'box' (draw a box around the trace and sum without weights) or 'optimal' (weight using Horne 1986 methods).
correct_zero    False                                       # Bool. Whether to model the contaminating 0th order and subtract it from your data during extraction. Sometimes works, sometimes just adds lots of scatter.
subtract_contam False                                       # Bool. Whether to model the contaminating orders and subtract them from your trace during extraction. Sometimes works, sometimes just adds lots of scatter.
sens_correction False                                       # Bool. Whether to correct for the G280's changing sensitivity as a function of wavelength. Since absolute calibrated spectra aren't needed in exoplanetary sciences, you can skip this safely.
mask_objs       [[0,0,0],]                                  # List of lists. If there are background objects in your planned aperture, mask them here. Each entry is (x,y,radius).

# Step 3a: Box extraction parameters
determine_hw    False                                       # Bool. If True, automatically determines preferred half-width for each order by minimizing out-of-transit/eclipse residuals.
indices         ([0,15],[-15,-1])                           # Lst of lsts of int. If determine_hw, these are the indices used to estimate the out-of-transit/eclipse residuals.
halfwidths_box  (12,12)                                     # Lst of ints. The half-width of extraction aperture to use for each order. Input here is ignored if 'determine_hw' is True.

# Step 3b: Optimum extraction parameters
aperture_type   'median'                                    # Str. Type of aperture to draw. Options are 'median', 'polyfit', 'smooth', or 'curved_poly'.
halfwidths_opt  (20,20)                                     # Lst of ints. The half-width of extraction aperture to use for each order. For optimum extraction, you should make this big (>12 pixels at least). There is no 'preferred' half-width in optimum extraction due to the weights.

# Step 4: 1D spectral cleaning and aligning
outlier_sigma   5.0                                         # Float. Sigma at which to reject spectral outliers in time. Outliers are replaced with median of timeseries. Enter False to skip this step.
align           True                                        # Bool. If True, uses cross-correlation to align spectra to keep wavelength solution consistent.

# ENDPARSE
"""

# Now we write the contents of the config file out to a .hustle file.
with open(os.path.join(config_directory,'stage_2_input_config.hustle'), 'w') as f:
    f.write(hustle_stage_2_file)

In [ ]:
run_pipeline(config_files_dir=config_directory,
             stages=(2,))

Notice that despite the considerably larger aperture used here, the results are still very similar, showing how the optimal profile has knocked down the noise contribution considerably.

That's all for Stage 2! You can execute this Stage for any G280 time series observation you have reduced.